In [1]:
!pip install transformers datasets accelerate torch peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('YOUR_TOKEN')"

In [3]:
import os
print(os.listdir('/kaggle/input/'))

['llm-llama-finetuning']


In [4]:
import shutil
import os

for file in os.listdir('/kaggle/input/llm-llama-finetuning/'):
    src = '/kaggle/input/llm-llama-finetuning/' + file
    dest = '/kaggle/working/'
    shutil.copy(src, dest)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb
import torch

model_name = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

raw_datasets = load_dataset("json", data_files={
    "train": "train.json",
    "validation": "validation.json"
})

def preprocess_function(examples):
    input_text = str(examples["Accident Description"])
    
    response_text = str(examples.get("Emergency response", "")) + " " + str(examples.get("Lesson Learned", ""))
    
    return tokenizer(
        input_text,
        truncation=True,
        max_length=512,
        padding="max_length",
        text_target=response_text,
        return_tensors="pt"
    )
    
tokenized_datasets = raw_datasets.map(
    preprocess_function, 
    batched=True, 
    batch_size=32,
    remove_columns=raw_datasets["train"].column_names
)

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

  
print(find_all_linear_names(model))

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=find_all_linear_names(model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./Llama-3.1-8B-finetuned",
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=100,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
    report_to="none" 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

trainer.train()

model.save_pretrained("./Llama-3.1-8B-finetuned")
tokenizer.save_pretrained("./Llama-3.1-8B-finetuned")


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

['down_proj', 'gate_proj', 'v_proj', 'k_proj', 'q_proj', 'o_proj', 'up_proj']


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/1955469765.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/pytho

Step,Training Loss,Validation Loss


('./Llama-3.1-8B-finetuned/tokenizer_config.json',
 './Llama-3.1-8B-finetuned/special_tokens_map.json',
 './Llama-3.1-8B-finetuned/tokenizer.json')